<font size="4" face="verdana" color="red"> Brainery Byte
<hr></font>
<font size="6" face="verdana" color="blue">Sections in OpenSees: <br>
    <b>MOMENT-CURVATURE & P-M INTERACTION </b><br>
    <br>
<br>
Standard W Section
</font><p>

<font size="4" face="verdana" color="black">
 R210704 <br>
 Silvia Mazzoni, 2021 <br>
      silviamazzoni@yahoo.com <br>

<br>
NOTES:
    <ol>
        <li> TENSION IS POSITIVE, COMPRESSION IS NEGATIVE, throughout!
        <li> This script takes advantage of dictionaries as a data structure to store model info. Using this apparent extra step allows you to store the information of your model in case you need to use it in post-processing. For example, it allows you to draw the fiber sections</li>
 <li>While this implementation uses a Standard AISC W section and RC Sections as examples, you can replace the section-defining module with your own section definition.</li>
        <li>Yes this WebApp uses function eSEESminiPy package to make the tool easier to use</li>
    </ol>
</font>
    
<BR>
<font size="4" face="verdana" color="black">
    <b>Process Outline:</b>
<ol type="1">
  <li>Initialize Pyton -- add necessary libraries and notebook setup</li>
  <li>Initialize javascript
  <li>Define Units (optional, but handy)</li>
  <li>Initialize Arrays Variables</li>
  <li>Define python utilities: define functions that make programming easier</li>
  <li>Define Normalized Loading Cycles for Material and Section Testing</li>
  <li>Start up OpenSees</li>
    <br>

  <li>For each Fiber section:</li>
    <ol type = "a">
      <li>Define Fiber Section (P-M-M interaction + Torsion)</li>
          <ol>
          <li>Initialize Section data and arrays</li>
          <li>Define Materials</li>
          <li>Test Materials</li>
          <li>Define Fiber geometry and material</li>
          <ol>
              <li>Define physical material fibers via patches, layers and fibers</li>
              <li>Define zero-area fibers for recorders</li>
          </ol>
          <li>Draw fiber section for visualization</li>
          <li>Define fiber Section in OpenSees</li>
        </ol>
      <li>Define Aggregated Section </li>
        <ol> 
            <li>define Vx and Vy behaviors </li>
            <li>aggregate Vx and Vy to Fiber Section in OpenSees using the '-section' option</li>
        </ol>
    </ol>
    <br>
          <li> For each Loading directin (z,y):</li>
          <ol>
              <li>Test Moment-Curvature response of Fiber Section at an axial load</li>
              <li> P-M interaction Analysis: Moment-Curvature response at a series of axial loads
          </ol>
</ol>
</font>
    
When you are done, you can print the notebook. You can also download the notebook to your local computer.

Because we are working in Binder, and Binder sessions are meant to be ephemeral, it is not possible for you to save any changes you make to your Jupyter Notebook. If you do make changes or notes, you will need to download the notebook to your own computer by clicking File > Download as > Notebook (.ipynb). The only way you will be able to run these is if you have the appropriate software to run Jupyter Notebooks in Python and pip install OpenSeesPy and eSEESminiPy in your Python configuration. You may view my videos on how to install Anaconda, Jupyter Notebooks and OpenSeesPy (https://www.youtube.com/c/silviasbrainery).
    
This Code has been developed by Silvia Mazzoni. Please acknowledge this in your scripts, when applicable

In [ ]:
# Initalize Python Libraries
import openseespy.opensees as ops
import eSEESminiPy
import matplotlib.pyplot as plt
import math

In [ ]:
%%javascript
// expand output box so graphs are viewed in notebook properly
IPython.OutputArea.auto_scroll_threshold = 1000;

In [ ]:
# define UNITS ----------------------------------------------------------------------------
inch  = 1.  # define basic units -- output units
kip  = 1. # define basic units -- output units
sec  = 1. # define basic units -- output units

LunitTXT  = 'inch' # define basic-unit text for output
FunitTXT  = 'kip' # define basic-unit text for output
TunitTXT  = 'sec' # define basic-unit text for output

ft  = 12.*inch # define engineering units
ksi  = kip/math.pow(inch,2)
psi  = ksi/1000.
lbf  = psi*inch*inch # pounds force
pcf  = lbf/math.pow(ft,3) # pounds per cubic foot
psf  = lbf/math.pow(ft,3) # pounds per square foot
inch2  = inch*inch # inch^2
inch4  = inch*inch*inch*inch # inch^4
cm  = inch/2.54 # centimeter, needed for displacement input in MultipleSupport excitation
PI  = 2*math.asin(1.0) # define constants
g  = 32.2*ft/math.pow(sec,2) # gravitational acceleration
Ubig  = 1.e10 # a really large number
Usmall  = 1/Ubig # a really small number
inchInv = 1/inch;


In [ ]:
# Initialize Arrays and variables
global matTag
matTag = 0
secTag = 0
SectionTagMap = {}
MaterialTagMap = {}
allSectionData = {}
WsectionData = {}
MomCurvData = {}
MatStressStrainData = {}
SectionLabelList = []
plt.close('all')

# Define bending directions for moment-curvature analysis
# you may remove either y or z if needed.
BendingDirectionList = ['z','y']

In [ ]:
# UTILITIES
# Define some local handy procs
def addMaterial(MaterialType,InputArray):
    global matTag
    matTag += 1
    ops.uniaxialMaterial(MaterialType,matTag,*InputArray)
    return matTag

### Plot Moment-Curvature Responses
def plotMomCurv(testSectionLabel,MomCurvData):
    thisFig = {}
    axModel = {}
    thisFigMomCurv = plt.figure('Moment Curvature ' + testSectionLabel,figsize=(6,3), dpi=200, facecolor='w', edgecolor='k' )
    iAx = 0
    for BendingDirection in MomCurvData[testSectionLabel].keys():
        iAx += 1
        axMomCurv = thisFigMomCurv.add_subplot(1,len(MomCurvData[testSectionLabel].keys()),iAx)
        line, = axMomCurv.plot(MomCurvData[testSectionLabel][BendingDirection]['Curvature'], MomCurvData[testSectionLabel][BendingDirection]['Moment'])
        eSEESminiPy.formatAx(axMomCurv,'Moment-Curvature ' + testSectionLabel+ ' ' + BendingDirection,'Curvature ' + BendingDirection,'Moment ' + BendingDirection,6,6)
        for thisFiberLabel in MomCurvData[testSectionLabel][BendingDirection]['StrainData'][SectionTagMap[testSectionLabel]].keys():
            if iAx == 1:
                thisFig[thisFiberLabel] = plt.figure('Stress Strain ' + thisFiberLabel,figsize=(4,1), dpi=200, facecolor='w', edgecolor='k' )
            axModel[thisFiberLabel] = thisFig[thisFiberLabel].add_subplot(1,len(MomCurvData[testSectionLabel].keys()),iAx)
            line, = axModel[thisFiberLabel].plot(MomCurvData[testSectionLabel][BendingDirection]['StrainData'][SectionTagMap[testSectionLabel]][thisFiberLabel],MomCurvData[testSectionLabel][BendingDirection]['StressData'][SectionTagMap[testSectionLabel]][thisFiberLabel], 'k',linewidth = 1)
            eSEESminiPy.formatAx(axModel[thisFiberLabel],'Stress Strain ' + thisFiberLabel + " " + BendingDirection,'Strain ' + BendingDirection,'Stress ' + BendingDirection,6,6)
            
# Plot P-M Interaction Data
def plotPMInterax(testSectionLabel,PMinteraxData):
    FiberFig = {}
    FiberAx = {}
    thisFigPMI = plt.figure('P-M Interax ' + testSectionLabel,figsize=(6,3), dpi=200, facecolor='w', edgecolor='k' )
    thisFigMomCurv = plt.figure('Moment Curvature ' + testSectionLabel,figsize=(6,3), dpi=200, facecolor='w', edgecolor='k' )
    iAx = 0
    for BendingDirection in BendingDirectionList:
        iAx += 1
        axPMI = thisFigPMI.add_subplot(1,2,iAx)
        axMomCurv = thisFigMomCurv.add_subplot(1,2,iAx)

        for CurvatureSign in PMinteraxData[testSectionLabel][BendingDirection].keys():
            for thisAxial in  PMinteraxData[testSectionLabel][BendingDirection][CurvatureSign].keys():
                thisMomCurvData = PMinteraxData[testSectionLabel][BendingDirection][CurvatureSign][thisAxial]
                #line, = axPMI.plot(thisMomCurvData['Moment'], thisMomCurvData['AxialForce'])
                axPMI.plot(max(thisMomCurvData['Moment'], key=abs), thisAxial,'ro',markerSize = 2,linewidth = 1)
                line, = axMomCurv.plot(thisMomCurvData['Curvature'], thisMomCurvData['Moment'])
                StrainData = thisMomCurvData['StrainData'][SectionTagMap[testSectionLabel]]
                StressData = thisMomCurvData['StressData'][SectionTagMap[testSectionLabel]]
                for thisFiberLabel,thisFiberStrain in StrainData.items():
                    if not thisFiberLabel in FiberFig.keys():
                        FiberFig[thisFiberLabel]= plt.figure('Fiber SS ' + testSectionLabel + thisFiberLabel,figsize=(4,1), dpi=200, facecolor='w', edgecolor='k' )
                    if not thisFiberLabel+ ' ' + BendingDirection in FiberAx.keys():
                        FiberAx[thisFiberLabel+ ' ' + BendingDirection] = FiberFig[thisFiberLabel].add_subplot(1,2,iAx)
                    thisFiberStress = StressData[thisFiberLabel]
                    line, = FiberAx[thisFiberLabel+ ' ' + BendingDirection].plot(thisFiberStrain,thisFiberStress,linewidth = 1)
        eSEESminiPy.formatAx(axPMI,'P-M Interax ' + testSectionLabel + ' ' + BendingDirection,'Bending Moment ' + BendingDirection,'Axial Force',6,6)
        eSEESminiPy.formatAx(axMomCurv,'P-M Interax MomentCurv ' + testSectionLabel + ' ' + BendingDirection ,'Curvature ' + BendingDirection,'Bending Moment ' + BendingDirection,6,6)
        eSEESminiPy.reverseYaxis(axPMI) # plot Axial Force Compression up
    for thisAxLabel,thisValue in FiberAx.items():
        eSEESminiPy.formatAx(thisValue,thisAxLabel,'Strain','Stress',6,6)
    plt.show()            

In [ ]:
# Define Normalized Loading Cycles for Material and Section Testing.

# These cycles are normalized to have a maximum value of 1.0
# You can later change the amplitude factor depending on the application
peaksArray = [0.1,0.2,0.3,0.5,0.7,0.85,1.0]
nCycles = 3
nSteps = 200
StrainSeries = eSEESminiPy.defineStrainHistory(peaksArray,nCycles,nSteps)
eSEESminiPy.plotYoneList(StrainSeries,title='Strain History',Ylabel='Strain')

In [ ]:
# Start-up OpenSees
ops.wipe()
Dim = '3D'
ops.model('basic','-ndm',3,'-ndf',6)

############################################################################################################<p>
<font size="6" face="verdana" color="blue">Fiber Section, Steel W Section <br></font>
############################################################################################################

In [ ]:
################################################################################################
### STEEL W SECTION
MainSectionLabel = 'W14X193'
################################################################################################

thisSectionType = '_FiberOnly'
thisSectionLabel = MainSectionLabel + thisSectionType

# initialize Section Data
# look up properties for W section
thisSectionData = eSEESminiPy.getWsectionData(MainSectionLabel)
thisMaterialTagMap = {}

# DEFINE MATERIAL
# -----------------------------------
# Define Steel Material
Fy=50*ksi
YoungsModulus = 2.900E+04*ksi
Bsh = 0.01
nu  = 0.3
ShearModulus  = YoungsModulus/2./(1+nu) # Torsional stiffness Modulus

# Backbone points for Hysteretic Material
f1 = Fy
eps1 = Fy/YoungsModulus
eps2 = 20*eps1
f2 = f1+Bsh*(eps2-eps1)*(YoungsModulus)
f3 = 0.9*f2
eps3 = 50*eps1
PositiveEnvelopeData = [f1,eps1,f2,eps2,f3,eps3]
NegativeEnvelopeData = [-f1,-eps1,-f2,-eps2,-f3,-eps3]

# hyteretic-material properties
[pinchX,pinchY] = [1,1]
[damage1,damage2] = [0.0,0.0]
beta = 0.5
hystereticData = [pinchX,pinchY,damage1,damage2,beta]
thisMaterialTagMap['Steel'] = addMaterial('Hysteretic',[*PositiveEnvelopeData,*NegativeEnvelopeData,*hystereticData])

thisMatStressStrainData = {}
ampFactor = 2*eps2
thisMatStressStrainData['Steel'] = eSEESminiPy.getMaterialStressStrain(Dim,thisMaterialTagMap['Steel'],StrainSeries,ampFactor)
eSEESminiPy.plotXYoneList(thisMatStressStrainData['Steel']['strain'],thisMatStressStrainData['Steel']['stress'],'Steel')

MaterialTagMap[thisSectionLabel] = thisMaterialTagMap
MatStressStrainData[thisSectionLabel] = thisMatStressStrainData


In [ ]:
# FIBER SECTION properties
# assign a section Tag for OpenSees and save it in the data map
secTag += 1
SectionTagMap[thisSectionLabel] = secTag
thisSectionData['SectionTag'] = secTag

d = eval(thisSectionData['D'])
bf = eval(thisSectionData['Bf'])
tf = eval(thisSectionData['Tf'])
tw = eval(thisSectionData['Tw'])
J = eval(thisSectionData['Ixx'])
dw  = d-2 * tf
GJ = ShearModulus*J
thisSectionData['GJ'] = GJ

# maximum compression capacity of section
thisSectionData['PmaxC'] = -0.99*eval(thisSectionData['A'])*Fy
thisSectionData['PmaxT'] = 0.99*eval(thisSectionData['A'])*Fy


# define fiber section data
FiberSectionCommandData = {}
FiberSectionCommandData['sectionCall'] = ['section','Fiber',secTag,'-GJ',GJ]
[nfdw,nftw,nfbf,nftf] = [8,2,16,2]
# key coordinates in W section -- y is vertical-up, z is horizontal-left
[y1,y2,y3,y4] = [-d/2,-dw/2,dw/2,d/2]
[z1,z2,z3,z4] = [-bf/2,-tw/2,tw/2,bf/2]
matID = thisMaterialTagMap['Steel']
# define properties of patches following opensees input format
FiberSectionCommandData['BottomFlange'] = ['patch','quad',matID,nfbf,nftf,y1,z4,y1,z1,y2,z1,y2,z4]
FiberSectionCommandData['Web'] = ['patch','quad',matID,nftw,nfdw,y2,z3,y2,z2,y3,z2,y3,z3]
FiberSectionCommandData['TopFlange'] = ['patch','quad',matID,nfbf,nftf,y3,z4,y3,z1,y4,z1,y4,z4]

# Add recorder fibers (zero area) and define strain limit states
strainLimits = {}
strainLimits['Yield'] = [-1.1*eps1,1.1*eps1]
strainLimits['LS'] = [-2*eps1,2*eps1]
strainLimits['CP'] = [-4*eps1,4*eps1]
FiberSectionCommandData['BottomLeft'] = ['fiber',y1,z4,0,matID,strainLimits]
FiberSectionCommandData['BottomRight'] = ['fiber',y1,z1,0,matID,strainLimits]
FiberSectionCommandData['TopRight'] = ['fiber',y4,z1,0,matID,strainLimits]
FiberSectionCommandData['TopLeft'] = ['fiber',y4,z4,0,matID,strainLimits]
FiberSectionCommandData['Origin'] = ['fiber',0,0,0,matID,strainLimits]

#print("OpenSees Input:")
#print(FiberSectionCommandData)

# draw Fiber Section:
eSEESminiPy.drawFiberSection(FiberSectionCommandData,thisSectionLabel)

# buildSection:
ops.section(*FiberSectionCommandData['sectionCall'][1:])
for thisFiberLabel,thisFiberCommandData in FiberSectionCommandData.items():
    fiberType = thisFiberCommandData[0]
    if fiberType == 'patch':
        ops.patch(*thisFiberCommandData[1:12+1])
    elif fiberType == 'layer':
        if thisFiberCommandData[3]>0:
            ops.layer(*thisFiberCommandData[1:8+1])
    elif fiberType == 'fiber':
        ops.fiber(*thisFiberCommandData[1:4+1])


thisSectionData['FiberSectionCommandData'] = FiberSectionCommandData
allSectionData[thisSectionLabel] = thisSectionData


In [ ]:
## RUN Moment-Curvature Analysis
testSectionLabel = MainSectionLabel + thisSectionType
MomCurvData[testSectionLabel] = {}

# The InputSection is a single argument to the Moment-Curvature Analysis
# If you want strain-recorder data, include the FiberSectionDommandData into a list:
FiberSectionCommandData = allSectionData[testSectionLabel]['FiberSectionCommandData']
InputSection = [SectionTagMap[testSectionLabel],FiberSectionCommandData]

### MOMENT-CURVATURE ANALYSIS
axialLoad = 180
ampFactor = 0.01
thisLimitState = '' # no limit state as we are doing a cyclic analysis
for BendingDirection in BendingDirectionList:
    MomCurvData[testSectionLabel][BendingDirection] = eSEESminiPy.runMomentCurvature(InputSection,axialLoad,StrainSeries,5,Dim+BendingDirection,ampFactor,thisLimitState)
    


In [ ]:
### Plot Moment-Curvature Responses
plotMomCurv(testSectionLabel,MomCurvData)
    

In [ ]:
# perform P-M Interaction Analysis for limit state
testSectionLabel = MainSectionLabel + thisSectionType

KappaMax = 0.005
ampFactor = 1.0
thisLimitState = 'LS'

# Define Series of Axial Loads
PmaxC = allSectionData[testSectionLabel]['PmaxC']
PmaxT = allSectionData[testSectionLabel]['PmaxT']
NaxialPointsC = 20
NaxialPointsT = 10
PaxialList = []
for i in range(NaxialPointsC):
    PaxialList.append(PmaxC - i*PmaxC/NaxialPointsC)
for i in range(NaxialPointsT+1):
    PaxialList.append(i*PmaxT/NaxialPointsT)

PMinteraxData = {}
PMinteraxData[testSectionLabel] = {}
for BendingDirection in BendingDirectionList:
    PMinteraxData[testSectionLabel][BendingDirection] = {}
    for CurvatureSign in [+1,-1]:
        PMinteraxData[testSectionLabel][BendingDirection][CurvatureSign] = {}
        for thisAxial in PaxialList:
            PMinteraxData[testSectionLabel][BendingDirection][CurvatureSign][thisAxial] = eSEESminiPy.runMomentCurvature([SectionTagMap[testSectionLabel],FiberSectionCommandData],thisAxial,CurvatureSign*KappaMax,100,Dim+BendingDirection,ampFactor,thisLimitState)


In [ ]:
# Plot P-M Interaction Data
plotPMInterax(testSectionLabel,PMinteraxData)

############################################################################################################<p>
<font size="6" face="verdana" color="blue">Aggregated Fiber Section, Steel W Section <br></font>
############################################################################################################

In [ ]:
# 3D Aggregated Section for Fiber Section:
thisSectionType = '_3Daggregated'
thisSectionLabel = MainSectionLabel + thisSectionType

# initialize
thisSectionData = allSectionData[MainSectionLabel + '_FiberOnly']
thisMaterialTagMap = {}
secTag += 1
SectionTagMap[thisSectionLabel] = secTag
thisSectionData['SectionTag'] = secTag


# Shear-Stiffness properties
ShearAreaY = eval(thisSectionData['D'])*eval(thisSectionData['Tw'])
ShearAreaZ = 0.667*2*eval(thisSectionData['Tf'])*eval(thisSectionData['Bf'])

# Define uniaxialMaterials for uncoupled section behavior
thisMaterialTagMap = {}
thisMaterialTagMap['-section'] = SectionTagMap[MainSectionLabel + '_FiberOnly']
thisMaterialTagMap['Vy'] = addMaterial('Elastic',[ShearModulus*ShearAreaY])
thisMaterialTagMap['Vz'] = addMaterial('Elastic',[ShearModulus*ShearAreaZ])

# create input for Aggregator Section
SectionInputArray = []
for (thisLabel) in ['Vy','Vz']:
    SectionInputArray.append(thisMaterialTagMap[thisLabel])
    SectionInputArray.append(thisLabel)
SectionInputArray.extend(['-section',thisMaterialTagMap['-section']])        
print('OpenSees Input Data:' )
print(SectionInputArray)

ops.section('Aggregator', secTag, *SectionInputArray)


thisSectionData['SectionInputArray'] = SectionInputArray
# add to all
MaterialTagMap[thisSectionLabel] = thisMaterialTagMap
allSectionData[thisSectionLabel] = thisSectionData



In [ ]:
## RUN Moment-Curvature Analysis
testSectionLabel = MainSectionLabel + thisSectionType
MomCurvData[testSectionLabel] = {}

# The InputSection is a single argument to the Moment-Curvature Analysis
# If you want strain-recorder data, include the FiberSectionDommandData into a list:
FiberSectionCommandData = allSectionData[testSectionLabel]['FiberSectionCommandData']
InputSection = [SectionTagMap[testSectionLabel],FiberSectionCommandData]

### MOMENT-CURVATURE ANALYSIS
axialLoad = 180
ampFactor = 0.01
thisLimitState = 'CP' # no limit state as we are doing a cyclic analysis
for BendingDirection in BendingDirectionList:
    MomCurvData[testSectionLabel][BendingDirection] = eSEESminiPy.runMomentCurvature(InputSection,axialLoad,StrainSeries,5,Dim+BendingDirection,ampFactor,thisLimitState)
    


In [ ]:
### Plot Moment-Curvature Responses
plotMomCurv(testSectionLabel,MomCurvData)    


In [ ]:
## This Code has been developed by Silvia Mazzoni. Please acknowledge this in your scripts, when applicable